In [12]:
import pandas as pd
import numpy as np
from Reducción_Memoria import reduce_mem_usage
from Control_de_Daños import control_de_danos
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import joblib

In [13]:
pd.set_option('display.max_columns', None)

COLS_TARGET_ENC = ['trip_origin_city', 'trip_destination_city', 'pos', 'rt_cc', 'trip_rt', 'rt_group']
COLS_ONE_HOT = ['trip_type', 'channel']

SEED = 42
TEST_SIZE_GENERAL = 0.2
TEST_SIZE_PAIS = 0.2
DATA_DIR = 'DataRT/'

In [14]:
def limpiar_moneda(serie):
    """
    Convierte strings de moneda con formato '1.000,00' a float.
    Si ya es numérico, lo devuelve tal cual (evita el error de los notebooks 03/04).
    """
    if pd.api.types.is_numeric_dtype(serie):
        return serie

    # Convierte a string, quita puntos de mil, cambia coma por punto
    clean = serie.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return pd.to_numeric(clean, errors='coerce').fillna(0)


In [15]:
def preparacion_maestra(df_input):
    """
    Aplica todas las reglas de negocio, limpieza de fechas y creación de target.
    """
    df = df_input.copy()

    # 1. Imputación de Nulos Lógicos
    cols_zero = ['length_of_stay', 'has_weekends', 'working_days']
    for col in cols_zero:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # 2. Limpieza de Monedas
    cols_dinero = ['ancillary_revenue', 'fare_revenue']
    for col in cols_dinero:
        if col in df.columns:
            df[col] = limpiar_moneda(df[col])

    # 3. Creación del TARGET (Regla de Negocio)
    if 'motivo_de_viaje' in df.columns:
        mapa = {'Trabajo': 1, 'Vacaciones': 0, 'Familiar': 0}
        df['target'] = df['motivo_de_viaje'].map(mapa)

    # 4. Ingeniería de Fechas (Cíclicas)
    if 'trip_start_date' in df.columns:
        fechas = pd.to_datetime(df['trip_start_date'], dayfirst=True, errors='coerce')

        # Ciclo Mensual (1-12)
        df['month_sin'] = np.sin(2 * np.pi * fechas.dt.month / 12)
        df['month_cos'] = np.cos(2 * np.pi * fechas.dt.month / 12)

        # Ciclo Semanal (0-6)
        df['dow_sin'] = np.sin(2 * np.pi * fechas.dt.dayofweek / 7)
        df['dow_cos'] = np.cos(2 * np.pi * fechas.dt.dayofweek / 7)

        df.drop(columns=['trip_start_date'], inplace=True)

    # 5. Eliminación de Columnas Ruidosas o Redundantes
    cols_drop = ['has_promo_class', 'promocode', 'recordlocator', 'paxs','discounted_fare_revenue_dc', 'discounted_fare_revenue_pc', 'discount_perc', 'infants', 'children', 'status', 'booking_hour', 'booking_dow', 'booking_weeknumber', 'booking_weeknumber', 'bookingid', 'working_days', 'weekend_days', 'booking_date', 'trip_end_dow', 'motivo_de_viaje', 'total_revenue']

    df.drop(columns=[c for c in cols_drop if c in df.columns], inplace=True, errors='ignore')
    print(f"Columnas eliminadas: {len(cols_drop)}")


    return df

In [16]:
df = pd.read_csv('DataSet.csv', sep=';')

In [17]:
# B. Control de Daños (Tu función original)
control_de_danos(df)



Porcentaje de NaN por columna:
                            Total_values  NaN_values  Completeness  \
promocode                          30455       28352          2103   
has_weekends                       30455       14652         15803   
length_of_stay                     30455       14645         15810   
trip_end_dow                       30455       14645         15810   
working_days                       30455       14645         15810   
weekend_days                       30455       14645         15810   
bookingid                          30455           0         30455   
recordlocator                      30455           0         30455   
booking_dow                        30455           0         30455   
booking_hour                       30455           0         30455   
motivo_de_viaje                    30455           0         30455   
pos                                30455           0         30455   
rt_group                           30455           0      

In [18]:
# C. Aplicar Preparación Maestra
df_clean = preparacion_maestra(df)


Columnas eliminadas: 21


In [19]:
# D. Reducción de Memoria Final
control_de_danos(df_clean)
df_clean = reduce_mem_usage(df_clean)



Porcentaje de NaN por columna:
                       Total_values  NaN_values  Completeness  missing_pct
pos                           30455           0         30455          0.0
rt_group                      30455           0         30455          0.0
channel                       30455           0         30455          0.0
days_to_departure             30455           0         30455          0.0
rt_cc                         30455           0         30455          0.0
trip_rt                       30455           0         30455          0.0
trip_origin_city              30455           0         30455          0.0
trip_destination_city         30455           0         30455          0.0
trip_type                     30455           0         30455          0.0
length_of_stay                30455           0         30455          0.0
trip_start_dow                30455           0         30455          0.0
has_weekends                  30455           0         30455       

In [20]:
# Identificar columnas categóricas (excluyendo 'pos' si se usa para segmentar)
# Asumo que 'pos' y otras de tipo 'object' o 'category' son las que quieres codificar.
df_clean_general = df_clean.copy()

# Separar X e y ANTES de codificar
X_general = df_clean_general.drop('target', axis=1)
y_general = df_clean_general['target']

# Dividir en train/test
X_train_gen, X_test_gen, y_train_gen, y_test_gen = train_test_split(
    X_general, y_general, test_size=TEST_SIZE_GENERAL, random_state=SEED, stratify=y_general
)

preprocessor_general = ColumnTransformer(
    transformers=[
        ('target_encoder', ce.TargetEncoder(), COLS_TARGET_ENC),
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), COLS_ONE_HOT)
    ],
    remainder='passthrough', # Mantiene las columnas no especificadas (numéricas)
    verbose_feature_names_out=False
).set_output(transform="pandas")

# Ajustar el preprocesador SOLO con los datos de entrenamiento
preprocessor_general.fit(X_train_gen, y_train_gen)

# Transformar ambos conjuntos
X_train_gen_encoded = preprocessor_general.transform(X_train_gen)
X_test_gen_encoded = preprocessor_general.transform(X_test_gen)

# Guardar archivos
X_train_gen_encoded.to_parquet(f'{DATA_DIR}X_train_general_encoded.parquet')
X_test_gen_encoded.to_parquet(f'{DATA_DIR}X_test_general_encoded.parquet')
y_train_gen.to_pickle(f'{DATA_DIR}y_train_general.pkl')
y_test_gen.to_pickle(f'{DATA_DIR}y_test_general.pkl')
joblib.dump(preprocessor_general, f'{DATA_DIR}preprocessor_general.pkl')

['DataRT/preprocessor_general.pkl']

In [21]:
# --- 2. PROCESAMIENTO PARA MODELOS POR PAÍS ---
print("\nProcesando y guardando para Modelos por País...")
paises = ['DOMCL', 'DOMAR', 'DOMPE', 'DOMCO', 'INTER']

for pais in paises:
    df_pais = df_clean[df_clean['rt_group'] == pais].copy()
    if df_pais.empty:
        continue

    print(f"Procesando para: {pais}")

    # Columnas para este país (excluimos 'pos' ya que es constante)
    cols_target_pais = [c for c in COLS_TARGET_ENC if c != 'rt_group']
    cols_onehot_pais = COLS_ONE_HOT

    X_pais = df_pais.drop(['target', 'rt_group'], axis=1)
    y_pais = df_pais['target']

    X_train_pais, X_test_pais, y_train_pais, y_test_pais = train_test_split(
        X_pais, y_pais, test_size=TEST_SIZE_PAIS, random_state=SEED, stratify=y_pais
    )

    # Preprocesador específico para el país
    preprocessor_pais = ColumnTransformer(
        transformers=[
            ('target_encoder', ce.TargetEncoder(), cols_target_pais),
            ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cols_onehot_pais)
        ],
        remainder='passthrough',
        verbose_feature_names_out=False
    ).set_output(transform="pandas")

    # Ajustar y transformar
    preprocessor_pais.fit(X_train_pais, y_train_pais)
    X_train_pais_encoded = preprocessor_pais.transform(X_train_pais)
    X_test_pais_encoded = preprocessor_pais.transform(X_test_pais)

    # Guardar archivos
    X_train_pais_encoded.to_parquet(f'{DATA_DIR}X_train_{pais}_encoded.parquet')
    X_test_pais_encoded.to_parquet(f'{DATA_DIR}X_test_{pais}_encoded.parquet')
    y_train_pais.to_pickle(f'{DATA_DIR}y_train_{pais}.pkl')
    y_test_pais.to_pickle(f'{DATA_DIR}y_test_{pais}.pkl')
    joblib.dump(preprocessor_pais, f'{DATA_DIR}preprocessor_{pais}.pkl')



Procesando y guardando para Modelos por País...
Procesando para: DOMCL
Procesando para: DOMAR
Procesando para: DOMPE
Procesando para: DOMCO
Procesando para: INTER


In [22]:
df_clean.to_parquet(f'{DATA_DIR}df_full.parquet')